# NuclearPy - Segmentation

This interactive notebook describes a step-by-step pipeline to predict nuclear segments of cells from 
high-content images and quantify for immunochemistry (ICC) levels and for a variety of nuclear features.

To perform an automated analysis on a batch of experiments, please refer to the `1.2_batch_segmentation_master.ipynb` notebook for detailed instructions.

## Pre-requisite

Please make sure that `NuclearPy` is successfully installed on your local computer. Installation instructions can be found on `NuclearPy` [GitHub repository](https://github.com/f-hamidlab/nuclearpy).

## NuclearPy segmentation pipeline

### Preparation
<br>
We begin by importing the following Python modules:

In [ ]:
import nuclearpy.segmentation as ncp
import matplotlib.pyplot as plt

<br>
Next, create a variable that contains the path to the high-content images. In this example, we will use a representative image of induced neuronal cells stained with DAPI, Beta3-tubulin, RFP marker and Neurogenin2.

In [ ]:
path = "../data/sample_images/experiment1"

### Create Segmentador object
<br>
NuclearPy stores data from high-content images as a `Segmentador` class. This object can be initialized using
`ngt.Segmentador` function with `path` as the mandatory input.
<br><br>
By default, the outputs of the analysis will be saved in `path` but users can specify 
a different output directory using the `outdir` param.  
<br><br>
Also, users can specify to analyse all images or just a single image by toggling the `analyse_all`
flag between `True` or `False`.

In [ ]:
nps = ncp.Segmentador(path, outdir = None, analyse_all=True)

NuclearPy automatically detects the number and wavelengths of the channels of all high-content images from the experiment. However, we require users to annotate these channels with the proteins/markers that are being labelled.
<br><br>
This marker labelling is performed by the `ngs.set_channels()` function, which takes up two optional parameters:

```
channels : str list
            List of names of each channel in order. If None, function will request user input
marker : string
            Name of nuclear marker
```

<br><br>
The following code will assign the channels and nulear marker for our sample image:

In [ ]:
nps.set_channels(channels = ["DAPI","Beta3","RFP","Ngn"], marker = "DAPI")

If the optional paramters are omitted, the function will activate an interactive message that prompts user for inputs.

### Nuclear segmentation
<br>
NuclearPy segments nuclei using Celllpose (default) or DeepCell methods. 
<br><br>
For the Cellpose method, users can optionally specify the expected nucleus diameter to speed the segmentation process.


In [ ]:
nps.nuclear_segmentation(method = "cellpose", 
                         diameter = 30, 
                         gamma_corr = 0.25, 
                         dc_scaleCorr = 1.9,
                         GPU = True)

<br>
To verify the accuracy of the segmentation, users may run the following code block:

In [ ]:
for f in nps.data["files"]:
    fig = nps.show_segmentation(file = f)
    plt.show()

### Measuring nuclear features
<br>
Once the nuclei masks are determined, users can run `nuclear_features` and `add_nuclear_features` functions to measure different features of the nuclei. The `nuclear_features` quantify primary features which include:

- nuclear area
- nuclear perimeter
- circularity
- eccentricity
- solidity
- major axis length
- minor axis length
- axes ratio
- angle of nuclei axes
- intensity of nuclear marker

`add_nuclear_features` measure additional features that include:

- core, internal and external intensity of nuclear marker
- average and total intensity of ICC

**Note**: Be sure to execute `nuclear_features` prior to running `add_nuclear_features`.

In [ ]:
nps.nuclear_features()
nps.add_nuclear_features()

<br>
The next block of code are optional, and quantifies peak intensities of DNA foci and spatial entropy of the nuclus.
Measure DNA dots and DNA peaks. This is an additional nuclear feature, whose measurement takes time. This feature finds the number of high intensity DNA dots in each nucleus.

In [ ]:
nps.find_dna_peaks(box_size = 10, zoom_box_size = 200)
nps.find_dna_dots(zoom_box_size = 200)
nps.spatial_entropy(d = 5, zoom_box_size = 200)

### Categorizing cells expressing ICC markers
<br>
NuclearPy attempts to categorize cells based on the intensities of each ICC markers. This is performed by the `markerGroup` function and the number of groups can be specified by the `n_groups` param.

In [ ]:
nps.markerGroup(n_groups = 5)

### Exporting segmented data
<br>
Users may export the segmented arrays, channel information and all nuclear feature meaurements using the following set of codes. Nuclear feature measurements are exported as a comma-delimted dataframe (CSV).

In [ ]:
nps.saveArrays()
nps.saveChannelInfo()
nps.export_csv(filename = "nuclearpy_output.csv")